In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import * 

In [0]:
%sql
SELECT * FROM youtube_project.silver.sales

In [0]:
df_silver = spark.read.format('parquet')\
                  .option('inferSchema',True)\
                  .load('abfss://silver@saprojectyt.dfs.core.windows.net/carsales/sales.parquet')

df_silver.display()

In [0]:
 dbutils.widgets.text("incremental_flag",'0')

In [0]:
incremental_flag = dbutils.widgets.get("incremental_flag")
print(incremental_flag)

In [0]:
df_scr = df_silver.select('Date_ID').distinct()
df_scr.display()

In [0]:
if  spark.catalog.tableExists('youtube_project.gold.dim_Date'):
    df_sink = spark.sql(''' 
                        SELECT Dim_Date_key,Date_ID
                        FROM youtube_project.gold.dim_Date
                        ''')
        
else:
    df_sink = spark.sql(''' 
                        SELECT 1 as Dim_Date_key,Date_ID
                        FROM PARQUET.`abfss://silver@saprojectyt.dfs.core.windows.net/carsales/sales.parquet`
                        WHERE 1 = 0
                        ''')



In [0]:
df_filter = df_scr.join(df_sink, df_scr['Date_ID'] == df_sink['Date_ID'], 'left').select(df_scr['Date_Id'], df_sink['Dim_Date_Key'])

In [0]:
df_filter_old = df_filter.filter(col('Dim_Date_Key').isNotNull())
df_filter_old.display()


In [0]:
df_filter_new = df_filter.filter(col('Dim_Date_Key').isNull()).select(df_scr['Date_ID'])
df_filter_new.display()

In [0]:
if incremental_flag == '0':
    max_value = 1
else:
    max_value_df = spark.sql('SELECT MAX(Dim_Date_key) FROM youtube_project.gold.dim_Date')
    max_value = max_value_df.collect()[0][0] + 1

In [0]:
df_filter_new = df_filter_new.withColumn('Dim_Date_key',max_value+monotonically_increasing_id())

In [0]:
df_filter_new.display()

In [0]:
df_final = df_filter_old.union(df_filter_new)
df_final.display()

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('youtube_project.gold.dim_Date'):
    detla_tbl = DeltaTable.forPath(spark,'abfss://gold@saprojectyt.dfs.core.windows.net/dim_Date')

    detla_tbl.alias('trg').merge(df_filter.alias('src'), 'trg.Dim_Date_key = src.Dim_Date_key')\
            .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
            .execute()

else:
    df_final.write.format('delta')\
            .mode('overwrite')\
            .option('path','abfss://gold@saprojectyt.dfs.core.windows.net/dim_Date')\
            .saveAsTable('youtube_project.gold.dim_Date')

In [0]:
%sql
SELECT * FROM gold.dim_Date